In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import matplotlib, imgaug, cv2, time, re, math, random, sys, os, shutil
import matplotlib.pyplot as plt

ROOT_DIR = os.path.abspath("../")

sys.path.append(ROOT_DIR)  # To find local version of the library

from mmrcnn.config import Config
import tqdm
from mmrcnn import model as modellib,utils,visualize
from mmrcnn.model import log

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

DEFAULT_WEIGHTS = os.path.join(ROOT_DIR, "mobile_mask_rcnn_coco.h5")

from mmrcnn.visualize import display_instances
from mmrcnn.config  import Config
from mmrcnn.visualize import display_instances
from mmrcnn.utils import extract_bboxes
from mmrcnn.utils import compute_ap
from mmrcnn.model import load_image_gt
from mmrcnn.model import mold_image

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# CLASS_LIST = ['1','2','3','4','7']
CLASS_LIST = ['body']

In [ ]:
# with open('../../select from activecam/coco/train.txt', 'r') as f:
#     train = f.read().split('\n')[:-1]
# with open('../../select from activecam/coco/val.txt', 'r') as f:
#     test = f.read().split('\n')[:-1]

In [ ]:
# train,test=[],[]
# for i in [j.split('.')[0] for j in os.listdir('../data')]:
#     if i=='':
#         continue
#     if random.random()>0.8:
#         test.append(i)
#     else:
#         train.append(i)
# #     break

In [ ]:
len(test), len(train)

In [ ]:
# import pickle
# with open('test.pickle', 'wb') as handle:
#     pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('train.pickle', 'wb') as handle:
#     pickle.dump(train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# l=[]
# print(len(test))
# for i in test_set.image_ids:
#     original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(test_set, config, 
#                            i, use_mini_mask=False)
#     i = test_set.image_info[i]['id']
#     if gt_class_id[0]!=4:
#         continue
#     if random.random()>0.5:
#         l.append(i)
# test=list(set(test)-set(l))
# print(len(test))

# l=[]
# print(len(train))
# for i in train_set.image_ids:
#     original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(train_set, config, 
#                            i, use_mini_mask=False)
#     i = train_set.image_info[i]['id']
#     if gt_class_id[0]!=4:
#         continue
#     if random.random()>0.5:
#         l.append(i)
# train=list(set(train)-set(l))
# print(len(train))

In [ ]:
# train

In [ ]:
# plt.hist(l)

In [ ]:
#  random.random()

In [ ]:
shutil.rmtree('../images')
os.mkdir('../images/')
for i in test+train:
    pic = plt.imread('../../raw/imported/{}.jpg'.format(re.sub('_','/',i)))
    pic  = cv2.resize(pic, (256,256), cv2.INTER_AREA)
    matplotlib.image.imsave('../images/{}.jpg'.format(i), pic)

In [ ]:
# traina

In [ ]:
from mmrcnn.utils import Dataset
class MyDataset(Dataset):
    def load_dataset(self, data):
        for i, cls in enumerate(CLASS_LIST):
            self.add_class("dataset", i, cls)
        for i in data:
                ann_path = os.path.join('../data/', i+'.xml')
                image_id = i.split('.')[0]
                with open(ann_path, 'r') as filetext:
                    s = filetext.read()
                    s = re.sub("^\s+|\n|\r|\s+$", '', s)
                    imname = re.findall('<filename>.*?</filename>', s)[0][7+3:-8-3]
                im_path = os.path.join('../images/', imname)
                self.add_image('dataset', image_id=image_id, path=im_path, annotation=ann_path)

    def extract_boxes(self, filename):
        with open(filename, 'r') as filetext:
            s = filetext.read()
            s = re.sub("^\s+|\n|\r|\s+$", '', s)
        boxes = list()
        imname = re.findall('<filename>.*?</filename>', s)[0][7+3:-8-3]
        width = int(int(re.findall('<width>.*?</width>', s)[0][7:-8]))
        height = int(int(re.findall('<height>.*?</height>', s)[0][8:-9]))
        for box in re.findall('<object>.*?</object>', s):
            name = re.findall('<name>.*?</name>', box)[0][6:-7]
            xmin = float(re.findall('<xmin>.*?</xmin>', box)[0][6:-7])
            ymin = float(re.findall('<ymin>.*?</ymin>', box)[0][6:-7])
            xmax = float(re.findall('<xmax>.*?</xmax>', box)[0][6:-7])
            ymax = float(re.findall('<ymax>.*?</ymax>', box)[0][6:-7])
            coors = [int(xmin/float(width)*256), 
                     int(ymin/float(height)*256), 
                     int(xmax/float(width)*256), 
                     int(ymax/float(height)*256), 
#                      name
                     'body'
                    ]
            boxes.append(coors)
        return boxes, 512, 512

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        path = info['annotation']
        boxes, w, h = self.extract_boxes(path)
        masks = np.zeros([h, w, len(boxes)], dtype='uint8')
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(str(box[4])))
        return masks, np.asarray(class_ids, dtype='int32')

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

train_set = MyDataset()
train_set.load_dataset(train) 
train_set.prepare()

test_set = MyDataset()
test_set.load_dataset(test) 
test_set.prepare()

print('Train: %d' % len(train_set.image_ids))
print('Test: %d' % len(test_set.image_ids))

In [ ]:
x,y=[],[]
for i in train_set.image_ids:
    image_id = i
    image = train_set.load_image(image_id)
    mask, class_ids = train_set.load_mask(image_id)
    bbox = extract_bboxes(mask)
    for b in bbox:
        x.append(b[2]-b[0])
        y.append(b[3]-b[1])
for i in test_set.image_ids:
    image_id = i
    image = test_set.load_image(image_id)
    mask, class_ids = test_set.load_mask(image_id)
    bbox = extract_bboxes(mask)
    for b in bbox:
        x.append(b[2]-b[0])
        y.append(b[3]-b[1])
plt.hist(x)
plt.hist(y)


In [ ]:
for i in range(2):
    image_id = i
    image = train_set.load_image(image_id)
    mask, class_ids = train_set.load_mask(image_id)
    bbox = extract_bboxes(mask)
    display_instances(image, bbox, mask, class_ids, train_set.class_names, show_mask=True)
for i in range(2):
    image_id = i
    image = test_set.load_image(image_id)
    mask, class_ids = test_set.load_mask(image_id)
    bbox = extract_bboxes(mask)
    display_instances(image, bbox, mask, class_ids, train_set.class_names, show_mask=True)

In [ ]:
class mConfig(Config):
    NAME = "mmrcnn_abovo"
    BATCH_SIZE = 4
    IMAGES_PER_GPU = 4
    GPU_COUNT = 1

    NUM_CLASSES = 1 + len(CLASS_LIST)  # COCO has 80 classes (1+80)

    BACKBONE = "mobilenetv1"

    BACKBONE_STRIDES = [4, 8, 16, 32, 64] #ResNet

    #RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256) #ResNet
    RPN_ANCHOR_SCALES = (75,100,125,150,175)

#     MINI_MASK_SHAPE = (56, 56) #ResNet
    MINI_MASK_SHAPE = (28, 28)

    IMAGE_MAX_DIM = 256
    IMAGE_MIN_DIM = 256

    #TRAIN_ROIS_PER_IMAGE = 200 #ResNet
    TRAIN_ROIS_PER_IMAGE = 128
    
    LOSS_WEIGHTS = {'rpn_class_loss': 1.0, 
                    'rpn_bbox_loss': 0.5, 
                    'mrcnn_class_loss': 1.0, 
                    'mrcnn_bbox_loss': 0.5, 
                    'mrcnn_mask_loss': 0.1}
    
    ROI_POSITIVE_RATIO = 0.6
    
    DETECTION_MAX_INSTANCES = 3
    MAX_GT_INSTANCES = 3
    
    STEPS_PER_EPOCH = len(train_set.image_ids)//BATCH_SIZE
    VALIDATION_STEPS = len(test_set.image_ids)//BATCH_SIZE
    
config = mConfig()
config.display()

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", model_dir = MODEL_DIR, config=config)
# model.keras_model.summary()

In [ ]:
# Which weights to start with?
init_weights = "coco"  # imagenet, coco, or last

if init_weights == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_weights == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(DEFAULT_WEIGHTS, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_weights == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [ ]:
import imageio
import imgaug
from imgaug import augmenters as iaa
np.random.bit_generator = np.random._bit_generator
seq = iaa.Sequential([
    iaa.Sometimes(0.5, [iaa.Fliplr(0.5)]),
    iaa.Sometimes(0.5, [iaa.Flipud(0.5)]),
    
    iaa.Sometimes(0.5, [iaa.Rot90(k=1)]),
    iaa.Sometimes(0.5, [iaa.Rot90(k=2)]),
    iaa.Sometimes(0.5, [iaa.Rot90(k=3)]),

    iaa.Sometimes(0.1, [iaa.Crop(px=(1, 16), keep_size=True)]),
    iaa.Sometimes(0.1, [iaa.GaussianBlur(sigma=(0, 3.0))]),
    iaa.Sometimes(0.9, [
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, 
            translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)}, 
            rotate=(-5, 5), 
            shear=(-2, 2),
            order=[0, 1], 
            cval=(0, 255), 
            mode=imgaug.ALL 
        )
    ]),
    iaa.Sometimes(0.5, [iaa.AdditiveGaussianNoise(scale=(1, 3))]),
    iaa.Sometimes(0.95, [iaa.LinearContrast(alpha=(0.7,1.3))]),
    iaa.Sometimes(0.05, [iaa.CLAHE(clip_limit=(1,5))]),
    iaa.Sometimes(0.1, [iaa.ElasticTransformation(alpha=9, sigma=9)]),
    iaa.Sometimes(0.01, [iaa.PiecewiseAffine(scale=(0,0.015))])
], random_order=True)

In [ ]:
# Training - Stage 1
# Finetune network heads
model.train(train_set, test_set,
            learning_rate=config.LEARNING_RATE,
            epochs=3,
            layers='heads',
            augmentation=seq)

In [ ]:
# model.train(train_set, test_set,
#             learning_rate=config.LEARNING_RATE,
#             epochs=6,
#             layers="11M+",
#             augmentation=seq)

In [ ]:
# Training - Stage 3
# Finetune all layers
model.train(train_set, test_set,
            learning_rate=config.LEARNING_RATE ,
            epochs=7,
            layers='all',
            augmentation=seq)

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mobile_mask_rcnn_abovo2.h5")
model.keras_model.save_weights(model_path)

In [ ]:
model.keras_model.save('mobile_mask_rcnn_abovo_allmodel2.h5')

In [ ]:
model.keras_model.get_config()

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file('mobile_mask_rcnn_abovo_allmodel2.h5')
tflite_model = converter.convert()

In [ ]:
# json_file = open('mobile_mask_rcnn_abovo_allmodel2.h5', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json)
# model.load_weights(args.model)

In [ ]:
class InferenceConfig(mConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
model.load_weights('../logs/512_mmrcnn_abovo20200326T0552/mask_rcnn_512_mmrcnn_abovo_0004.h5', by_name=True)

In [ ]:
# # Test on a random image
# image_id = random.choice(test_set.image_ids)
# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(test_set, inference_config, 
#                            image_id, use_mini_mask=False)

# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             test_set.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            test_set.class_names, r['scores'], ax=get_ax())

In [ ]:
print('here')

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(test_set.image_ids, 10)
APs = []
stat=[]
l=[]
for image_id in tqdm.tqdm(test_set.image_ids):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(test_set, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
#     print(gt_class_id, r['class_ids'])
#     print(AP,precisions, recalls)

    stat.append([gt_class_id, r['class_ids'], gt_bbox, r["rois"], r['scores']])


    l.append(len(r['class_ids']))
    
    APs.append(AP)
    
print("mAP: ", np.mean(APs))
stat = np.array(stat)

In [ ]:
plt.hist(l)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:
n=0
res=[]
for y,yhat,rbox,pbox,scores in stat:
    maxiou = 0
    cls = 7
    ssc = 0
    for box,i,sc in zip(pbox,yhat,scores):
#         print(box,rbox)
        iou = bb_intersection_over_union(box,rbox[0])
        if iou>maxiou and iou>0.5:
            maxiou = iou
            cls = i
            ssc=sc
#     print(y,cls,ssc)
    res.append([y[0],cls])
#     if n>10:
#         break

In [ ]:
yhat = np.array(res)[:,1]
real = np.array(res)[:,0]

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(list(real), list(yhat), labels=[1,2,3,4,5,6])
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); 
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels([1,2,3,4,5,7]); ax.yaxis.set_ticklabels([1,2,3,4,5,7]);

In [ ]:
l=[]
for i,j in res:
    l.append(i==j)
sum(l)/len(l)